Setting up the env, important to set the default type to futures

In [31]:

import pandas as pd
from ccxt import binance as binance_sync
from ccxt.pro import binance as binance_pro
from datetime import datetime

proxies = {}
# Initialize exchange instances
binance_exchange = binance_sync({
                'proxies': proxies,
                'sandbox': False,  # 使用实盘数据
                'enableRateLimit': True,
                'options': {
                    'defaultType': 'future',  # 使用期货市场
                }
            })
binance_live  = binance_pro({
                'proxies': proxies,
                'sandbox': False,  # 使用实盘数据
                'enableRateLimit': True,
                'options': {
                    'defaultType': 'future',  # 使用期货市场
                }
            })

perp = "ETH-USDT-PERP"
base = perp.split('-')[0]
perp = f"{base}/USDT"

limit = 1000
target_date = datetime(2025, 9, 13)  # September 11, 2025
timestamp = int(target_date.timestamp() * 1000)  
timeframe = "1m"
# Historical


In [32]:
import asyncio
from datetime import datetime

def convert_ohlcv_timestamps(ohlcv_data):
    """Convert OHLCV data timestamps to readable datetime format"""
    converted_data = []
    for candle in ohlcv_data:
        # Convert timestamp (first element) from milliseconds to datetime
        timestamp_ms = candle[0]
        dt = datetime.fromtimestamp(timestamp_ms / 1000)
        # Replace timestamp with readable datetime string
        converted_candle = [dt.strftime('%Y-%m-%d %H:%M:%S')] + candle[1:]
        converted_data.append(converted_candle)
    return converted_data


Historical Closes


In [33]:
ohlcv = binance_exchange.fetch_ohlcv(perp, timeframe, since=timestamp, limit=1000)
print("Historical data (first 5 candles):")
ohlcv_converted = convert_ohlcv_timestamps(ohlcv)
for i, candle in enumerate(ohlcv_converted[:5]):
    print(f"Candle {i+1}: {candle}")
print(f"... and {len(ohlcv_converted)-5} more candles")

Historical data (first 5 candles):
Candle 1: ['2025-09-13 00:00:00', 4693.35, 4695.21, 4691.0, 4691.01, 1652.423]
Candle 2: ['2025-09-13 00:01:00', 4691.0, 4692.4, 4689.68, 4689.68, 1841.625]
Candle 3: ['2025-09-13 00:02:00', 4689.66, 4690.0, 4688.58, 4688.91, 3936.343]
Candle 4: ['2025-09-13 00:03:00', 4688.92, 4690.0, 4687.75, 4687.76, 2606.01]
Candle 5: ['2025-09-13 00:04:00', 4687.76, 4688.09, 4686.05, 4686.31, 1499.675]
... and 995 more candles


We can read orderbook at any point

In [34]:
orderbook = binance_exchange.fetch_order_book(perp)
timestamp = orderbook['timestamp']
dt = datetime.fromtimestamp(timestamp / 1000).strftime('%Y-%m-%d %H:%M:%S UTC')
dt


'2025-09-14 12:36:44 UTC'

In [35]:
bids_df = pd.DataFrame(orderbook['bids'], columns=['Price', 'Amount']).sort_values(by='Price', ascending=False)
asks_df = pd.DataFrame(orderbook['asks'], columns=['Price', 'Amount']).sort_values(by='Price', ascending=True)


In [36]:
bids_df.head()
asks_df.head()


,Price,Amount
0,4638.46,163.060
1,4638.47,64.145
2,4638.48,35.036
3,4638.50,35.764
4,4638.52,0.100


Historical Trades - There is a delay

In [ ]:
trades = binance_exchange.fetch_trades(perp, limit=1000)
trades_df = pd.DataFrame(trades, columns=['timestamp', 'price', 'amount', 'side'])

# Convert timestamp to readable datetime
trades_df['datetime'] = pd.to_datetime(trades_df['timestamp'], unit='ms')

# Select key columns for analysis
key_columns = ['datetime', 'side', 'price', 'amount']
trades_df = trades_df[key_columns]

# Sort by timestamp
trades_df = trades_df.sort_values(by='datetime')

trades_df.head()



,datetime,side,price,amount
0,2025-09-14 11:35:29.080,buy,4637.43,0.008
1,2025-09-14 11:35:29.080,buy,4637.44,0.009
2,2025-09-14 11:35:29.080,buy,4637.45,0.421
3,2025-09-14 11:35:29.080,buy,4637.46,0.019
4,2025-09-14 11:35:29.080,buy,4637.48,0.018


Funding Rates - pay attention to the notation of the symbol, that it is a perp

In [38]:
recent_timestamp = int((datetime.now().timestamp() - 7*24*3600) * 1000)
funding_rates = binance_exchange.fetch_funding_rate_history("ETH/USDT:USDT", since=recent_timestamp, limit=100)
funding_rates_df = pd.DataFrame(funding_rates)
funding_rates_df['timestamp'] = pd.to_datetime(funding_rates_df['timestamp'], unit='ms')
funding_rates_df['mark_price'] = funding_rates_df['info'].apply(lambda x: float(x['markPrice']))
key_columns = ['timestamp', 'fundingRate', 'mark_price', "symbol"]
funding_clean = funding_rates_df[key_columns]
funding_clean

,timestamp,fundingRate,mark_price,symbol
0,2025-09-07 16:00:00.000,0.000033,4297.205047,ETH/USDT:USDT
1,2025-09-08 00:00:00.007,0.000026,4304.153891,ETH/USDT:USDT
2,2025-09-08 08:00:00.000,0.000021,4289.820000,ETH/USDT:USDT
3,2025-09-08 16:00:00.000,0.000091,4369.830000,ETH/USDT:USDT
4,2025-09-09 00:00:00.000,0.000047,4303.986659,ETH/USDT:USDT
5,2025-09-09 08:00:00.000,0.000100,4360.430000,ETH/USDT:USDT
6,2025-09-09 16:00:00.000,0.000066,4282.783357,ETH/USDT:USDT
7,2025-09-10 00:00:00.000,0.000062,4308.552543,ETH/USDT:USDT
8,2025-09-10 08:00:00.002,0.000088,4327.973539,ETH/USDT:USDT
9,2025-09-10 16:00:00.000,0.000100,4403.500000,ETH/USDT:USDT


Open Interest - the amount of derivates on the market


In [39]:

open_interest = binance_exchange.fetch_open_interest(perp)
open_interest


{'symbol': 'ETH/USDT:USDT',
 'baseVolume': 1980005.665,
 'quoteVolume': None,
 'openInterestAmount': 1980005.665,
 'openInterestValue': None,
 'timestamp': 1757849805437,
 'datetime': '2025-09-14T11:36:45.437Z',
 'info': {'symbol': 'ETHUSDT',
  'openInterest': '1980005.665',
  'time': '1757849805437'}}

Watch ticker - Provides latest info on prices - probably better for risk management

In [40]:
watch_ticker =await binance_live.watch_ticker(perp)
# Create DataFrame
ticker_df = pd.DataFrame([watch_ticker])

# Convert timestamp to readable datetime
ticker_df['datetime_readable'] = pd.to_datetime(ticker_df['timestamp'], unit='ms')

# Select and rename key columns
key_columns = ['datetime_readable', 'symbol', 'open', 'high', 'low', 'close', 'last', 'change', 'percentage', 'baseVolume', 'quoteVolume', 'vwap']
ticker_clean = ticker_df[key_columns].copy()

print("Ticker data:")
print(ticker_clean)


Ticker data:
        datetime_readable         symbol     open     high     low    close  \
0 2025-09-14 11:36:54.990  ETH/USDT:USDT  4715.99  4729.48  4603.0  4637.23   

      last  change  percentage   baseVolume   quoteVolume     vwap  
0  4637.23  -78.76       -1.67  2747503.442  1.280201e+10  4659.51  


Live Order Book

In [41]:
watch_order_book = await binance_live.watch_order_book(perp)
watch_order_book_df = pd.DataFrame(watch_order_book, columns=['timestamp', 'bids', 'asks'])
watch_order_book_df.head()

,timestamp,bids,asks
0,1757849816430,"[4637.23, 88.4]","[4637.24, 195.123]"
1,1757849816430,"[4637.22, 0.944]","[4637.25, 2.618]"
2,1757849816430,"[4637.21, 0.026]","[4637.26, 0.968]"
3,1757849816430,"[4637.2, 0.005]","[4637.27, 0.031]"
4,1757849816430,"[4637.19, 0.005]","[4637.31, 0.005]"


Live trades Freq - ANY

In [42]:
watch_trades = await binance_live.watch_trades(perp)
watch_trades_df = pd.DataFrame(watch_trades, columns=['timestamp', 'price', 'amount', 'side'])
watch_trades_df["timestamp"] = pd.to_datetime(watch_trades_df["timestamp"], unit="ms")
watch_trades_df.head()


,timestamp,price,amount,side
0,2025-09-14 11:36:58.322,4637.24,0.31,buy


Candle Data freq - Minute

In [43]:
watch_ohlcv = await binance_live.watch_ohlcv(perp)
watch_ohlcv_df = pd.DataFrame(watch_ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
watch_ohlcv_df["timestamp"] = pd.to_datetime(watch_ohlcv_df["timestamp"], unit="ms")
watch_ohlcv_df


,timestamp,open,high,low,close,volume
0,2025-09-14 11:36:00,4638.92,4638.92,4636.52,4637.23,1757.846


In [44]:
watch_ohlcv

[[1757849760000, 4638.92, 4638.92, 4636.52, 4637.23, 1757.846]]